In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.datasets import fetch_openml
import numpy as np


### Load Datasets

#### Load MNIST

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [6]:
x_train[0:64].shape

(64, 28, 28)

In [8]:
first_mnist_batch = x_train[0:64]
first_mnist_batch = first_mnist_batch.reshape(64, 28, 28, 1)
first_mnist_batch.shape

(64, 28, 28, 1)

### Load USPS

In [9]:
# Load the USPS dataset from OpenML
usps = fetch_openml('usps', version=1)

# Convert the data into a more usable form
usps_data = usps.data.astype(np.float32)
usps_labels = usps.target.astype(np.int32)

# Reshape the images to 16x16 (if necessary)
usps_data = usps_data.values.reshape(-1, 16, 16)

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:1030: UserWarning: Version 1 of dataset USPS is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://api.openml.org/data/v1/download/18805612/USPS.arff
  warn(


In [11]:
usps_data[0:64].shape

(64, 16, 16)

In [12]:
first_usps_batch = usps_data[0:64]
first_usps_batch = first_usps_batch.reshape(64, 16, 16, 1)
first_usps_batch.shape

(64, 16, 16, 1)

### Model Architecture

#### Encoders Architecture

In [21]:
enc_first_layer = models.Sequential([
    layers.InputLayer(shape=(None, None, 1)),

    layers.Conv2D(64, (7, 7), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

enc_second_layer = models.Sequential([
    layers.InputLayer(shape=(None, None, 64)),

    layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

enc_third_layer_source = models.Sequential([
    layers.InputLayer(shape=(None, None, 128)),

    layers.Conv2D(256, (3, 3), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

enc_third_layer_target = models.Sequential([
    layers.InputLayer(shape=(None, None, 128)),

    layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

enc_forth_layer_source = models.Sequential([
    layers.InputLayer(shape=(None, None, 256)),

    layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

enc_forth_layer_target = models.Sequential([
    layers.InputLayer(shape=(None, None, 256)),

    layers.Conv2D(512, (2, 2), strides=(1, 1), padding='valid'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

enc_fifth_layer =  models.Sequential([
    layers.InputLayer(shape=(None, None, 512)),

    layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

enc_sixth_layer = models.Sequential([
    layers.InputLayer(shape=(None, None, 256)),

    layers.Conv2D(128, (3, 3), strides=(3, 3), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

mnist_encoder = [enc_first_layer, enc_second_layer, enc_third_layer_source, enc_forth_layer_source, enc_fifth_layer, enc_sixth_layer]
usps_encoder = [enc_first_layer, enc_second_layer, enc_third_layer_target, enc_forth_layer_target, enc_fifth_layer, enc_sixth_layer]

#### Generators Architecture

In [22]:
gen_first_layer = models.Sequential([
    layers.InputLayer(shape=(None, None, 128)),
    layers.Conv2DTranspose(128, (3, 3), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

gen_second_layer = models.Sequential([
    layers.InputLayer(shape=(None, None, 128)),
    layers.Conv2DTranspose(256, (3, 3), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

gen_third_layer_source = models.Sequential([
    layers.InputLayer(shape=(None, None, 256)),
    layers.Conv2DTranspose(512, (3, 3), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

gen_third_layer_target = models.Sequential([
    layers.InputLayer(shape=(None, None, 256)),
    layers.Conv2DTranspose(512, (2, 2), strides=(1, 1), padding='valid'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

gen_forth_layer = models.Sequential([
    layers.InputLayer(shape=(None, None, 512)),
    layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

##################################################################

gen_fifth_layer_source =  models.Sequential([
    layers.InputLayer(shape=(None, None, 256)),
    layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

gen_fifth_layer_target=  models.Sequential([
    layers.InputLayer(shape=(None, None, 256)),
    layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])


##################################################################

gen_sixth_layer = models.Sequential([
    layers.InputLayer(shape=(None, None, 128)),

    layers.Conv2D(64, (7, 7), strides=(1, 1), padding='same'),
    layers.BatchNormalization(),
    layers.LeakyReLU(),
])

gen_last_layer = models.Sequential([
    layers.Conv2DTranspose(1, kernel_size=(1, 1), strides=(1, 1), padding="same"),
    layers.Activation("tanh")
])

mnist_generator = [gen_first_layer, gen_second_layer, gen_third_layer_source, gen_forth_layer, gen_fifth_layer_source, gen_sixth_layer, gen_last_layer]
usps_generator = [gen_first_layer, gen_second_layer, gen_third_layer_target, gen_forth_layer, gen_fifth_layer_target, gen_sixth_layer, gen_last_layer]

### Forward Passes

In [23]:
x = first_mnist_batch

mnist_latent = None
print(f'Start: {x.shape}')
i = 1
for layer in mnist_encoder:
  x = layer.predict(x)
  print(f'{i}th layer: {x.shape}')
  mnist_latent = x
  i += 1

print(f'End: {mnist_latent.shape}')

Start: (64, 28, 28, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1th layer: (64, 28, 28, 64)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
2th layer: (64, 14, 14, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
3th layer: (64, 7, 7, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
4th layer: (64, 7, 7, 512)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
5th layer: (64, 7, 7, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
6th layer: (64, 3, 3, 128)
End: (64, 3, 3, 128)


In [24]:
x = first_usps_batch

usps_latent = None
print(f'Start: {x.shape}')
i = 1
for layer in usps_encoder:
  x = layer.predict(x)
  usps_latent = x
  print(f'{i}th layer: {x.shape}')
  i += 1

print(f'End: {usps_latent.shape}')

Start: (64, 16, 16, 1)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1th layer: (64, 16, 16, 64)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2th layer: (64, 8, 8, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3th layer: (64, 8, 8, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
4th layer: (64, 7, 7, 512)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
5th layer: (64, 7, 7, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
6th layer: (64, 3, 3, 128)
End: (64, 3, 3, 128)


Both result in shape `(3, 3, 128)`, which seems good.

In [25]:
# generate from latent space usps image
x = mnist_latent
usps_gen_img = None
print(f'Start: {x.shape}')
i = 1
for layer in usps_generator:
  x = layer.predict(x)
  print(f'{i}th layer: {x.shape}')
  usps_gen_img = x
  i += 1

print(f'End: {usps_gen_img.shape}')

Start: (64, 3, 3, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step 
1th layer: (64, 3, 3, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
2th layer: (64, 3, 3, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
3th layer: (64, 4, 4, 512)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
4th layer: (64, 8, 8, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
5th layer: (64, 8, 8, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
6th layer: (64, 8, 8, 64)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step 
7th layer: (64, 8, 8, 1)
End: (64, 8, 8, 1)


USPS image should be of shape `(16, 16)`

In [26]:
# from latent to mnist
x = usps_latent
mnist_gen_img = None
print(f'Start: {x.shape}')
i = 1
for layer in mnist_generator:
  x = layer.predict(x)
  print(f'{i}th layer: {x.shape}')
  mnist_gen_img = x
  i += 1

print(f'End: {mnist_gen_img.shape}')

Start: (64, 3, 3, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1th layer: (64, 3, 3, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
2th layer: (64, 3, 3, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
3th layer: (64, 3, 3, 512)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
4th layer: (64, 6, 6, 256)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
5th layer: (64, 12, 12, 128)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
6th layer: (64, 12, 12, 64)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
7th layer: (64, 12, 12, 1)
End: (64, 12, 12, 1)


MNIST image should be of shape `(28, 28)`.

### Discriminators

In [ ]:
# First Layer
ld_first_layer = models.Sequential([
    layers.InputLayer(input_shape=(None, None, 128)),  # Assuming 3 channels (RGB input)
    layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same'),
    layers.LeakyReLU(alpha=0.1),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)),
])

##################################################################

# Second Layer
ld_second_layer = models.Sequential([
    layers.InputLayer(input_shape=(None, None, 256)),
    layers.Conv2D(512, (3, 3), strides=(1, 1), padding='same'),
    layers.LeakyReLU(alpha=0.1),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
])

##################################################################

# Third Layer
ld_third_layer = models.Sequential([
    layers.InputLayer(input_shape=(None, None, 512)),
    layers.Conv2D(256, (3, 3), strides=(1, 1), padding='same'),
    layers.LeakyReLU(alpha=0.1),
    layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1)),
])

##################################################################

# Fourth Layer (Fully connected)
# ld_fourth_layer = models.Sequential([
#     layers.InputLayer(input_shape=(None, None, 256)),  # Assuming the final output size from Conv layers
#     layers.Flatten(),  # Flatten the output to feed into the Dense layer
#     layers.Dense(1),
#     # layers.Activation("sigmoid")
# ])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
